# Ingress

* [kubernetes.io](https://kubernetes.io/docs/concepts/services-networking/ingress/)
* [NGINX Ingress Controller](https://kubernetes.github.io/ingress-nginx/)
* [kubespray docs](https://kubespray.io/#/docs/ingress_controller/ingress_nginx)
* [kubespray ansible files](https://github.com/kubernetes-sigs/kubespray/tree/331647f4abca7d059c248ac10b7ba43d9f4b86fc/roles/kubernetes-apps/ingress_controller/ingress_nginx)
* [Publishing Services (ServiceTypes)](https://kubernetes.io/docs/concepts/services-networking/service/#publishing-services-service-types)

As a reminder from the kubernetes docs:

> [Ingress is] An API object that manages external access to the services in a cluster, typically HTTP.

> Ingress may provide load balancing, SSL termination and name-based virtual hosting

> Ingress exposes HTTP and HTTPS routes from outside the cluster to services within the cluster. Traffic routing is controlled by rules defined on the Ingress resource.

> An Ingress controller is responsible for fulfilling the Ingress, usually with a load balancer, though it may also configure your edge router or additional frontends to help handle the traffic.

> An Ingress does not expose arbitrary ports or protocols. Exposing services other than HTTP and HTTPS to the internet typically uses a service of type Service.Type=NodePort or Service.Type=LoadBalancer.

## Goal

* Run a pod with a simple http server that is publicly accessible via **http://app1.foo**

## Questions

### Do I have to use a Cloud Manager controlled LoadBalancer or can I get it working with just the nginx ingress and some manuel configuration?

* [Use a Service to Access an Application in a Cluster](https://kubernetes.io/docs/tasks/access-application-cluster/service-access-application-cluster/)

### What's my Ingress Class?
see [Ingress Class](https://kubernetes.io/docs/concepts/services-networking/ingress/#ingress-class)?

### Who handles/terminates TLS ?

### Do I need a service type LoadBalancer?

See [service type LoadBalancer](https://kubernetes.io/docs/tasks/access-application-cluster/create-external-load-balancer/#create-a-service)? (This would mean I need a cloud supported load balancer or MetalLB...)

### What's the service type External IPs for?

[External IPs](https://kubernetes.io/docs/concepts/services-networking/service/#external-ips)

## Steps

* Install the nginx ingress controller via kubespray
* Deploy [an Ingress object](https://kubernetes.github.io/ingress-nginx/user-guide/basic-usage/)

## Install nginx ingress controller via kubespray

Looking at [the manifests](https://github.com/kubernetes-sigs/kubespray/tree/785324827cd8ad5c69c0d6af23549b4ae5c8bfbd/roles/kubernetes-apps/ingress_controller/ingress_nginx/templates) I expect the following to be installed:

* 1 clusterRole `ingress-nginx`
* 1 clusterRoleBinding `ingress-nginx`
* 1 PodSecurityPolicy `ingress-nginx`
* 1 namespace `ingress-nginx` and within that namespace:
    * 3 ConfigMaps
    * 1 DaemonSet
    * 1 Role
    * 1 RoleBinding
    * 1 ServiceAccount

In [5]:
! cd /usr/src/kubespray && ansible-playbook -i /usr/src/app/mycluster cluster.yml -t ingress-nginx -b -e ingress_nginx_enabled=true

### Verify installation

`kubectl get pods --all-namespaces -l app.kubernetes.io/name=ingress-nginx --watch`

In [13]:
! kubectl get ns

In [14]:
! kubectl get all -n ingress-nginx

In [15]:
! kubectl get cm -n ingress-nginx

In [16]:
! kubectl get sa -n ingress-nginx

In [21]:
! kubectl logs $(kubectl get po -n ingress-nginx -o jsonpath="{.items[0].metadata.name}") -n ingress-nginx

In [20]:
! kubectl get po -n ingress-nginx -o jsonpath="{.items[*].metadata.name}"

## Run a pod with a simple http server that is publicly accessible via http://app1.foo

In [24]:
! kubectl run app1 --image=nginx

In [33]:
! kubectl expose po app1 --port=80 --target-port=80

In [144]:
# get the ClusterIp of the app1 service and store in a pseudo env var
! echo $(kubectl get svc -l run=app1 -o jsonpath="{.items[0].spec.clusterIP}") > /tmp/APP1 && cat /tmp/APP1

10.233.16.147


In [146]:
# Verify app1 responds on its cluster ip
! ssh node1 curl http://$(cat /tmp/APP1)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 <!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>
100   615  100   615    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k


In [147]:
"""
Write the ingress manifest to a local file.
"""

content = """apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: app1
spec:
  rules:
  - host: app1.foo
    http:
      paths:
      - path: /
        pathType: Prefix
        backend:
          service:
            name: app1
            port:
              number: 80
  # ingressClassName: nginx
"""

with open('/tmp/app1-ingress.yml', 'w') as f:
    f.write(content)

In [70]:
! kubectl apply -f /tmp/app1-ingress.yml

ingress.networking.k8s.io/app1 created


In [75]:
! kubectl describe ingress app1

Name:             app1
Labels:           <none>
Namespace:        default
Address:          xxx.xxx.235.216,xxx.xxx.195.197
Ingress Class:    <none>
Default backend:  <default>
Rules:
  Host        Path  Backends
  ----        ----  --------
  app1.foo    
              /   app1:80 (10.233.92.3:80)
Annotations:  <none>
Events:
  Type    Reason  Age                From                      Message
  ----    ------  ----               ----                      -------
  Normal  Sync    16s (x2 over 51s)  nginx-ingress-controller  Scheduled for sync
  Normal  Sync    16s (x2 over 51s)  nginx-ingress-controller  Scheduled for sync


In [186]:
# get ingress1's ip
! echo $(kubectl get ingress -o jsonpath="{.items[0].status.loadBalancer.ingress[0].ip}") > /tmp/NODE1

In [185]:
# get ingress2's ip
! echo $(kubectl get ingress -o jsonpath="{.items[0].status.loadBalancer.ingress[1].ip}") > /tmp/NODE2

In [168]:
# Write ingress ips to /etc/hosts
! echo "$(cat /tmp/NODE1) app1.foo" >> /etc/hosts
! echo "$(cat /tmp/NODE2) app1.foo" >> /etc/hosts
! cat /etc/hosts

# <nerdctl>
127.0.0.1          localhost localhost.localdomain
:1                 localhost localhost.localdomain
10.4.0.31          1704616f7564 nginx
10.4.0.65          fcb3c336a3c9 kube-lab
# </nerdctl>

xxx.xxx.235.216 app1.foo
xxx.xxx.195.197 app1.foo


In [171]:
! curl -v http://app1.foo -I

*   Trying xxx.xxx.235.216:80...
* Connected to app1.foo (xxx.xxx.235.216) port 80 (#0)
> HEAD / HTTP/1.1
> Host: app1.foo
> User-Agent: curl/7.74.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
HTTP/1.1 200 OK
< Date: Fri, 10 Jun 2022 12:27:22 GMT
Date: Fri, 10 Jun 2022 12:27:22 GMT
< Content-Type: text/html
Content-Type: text/html
< Content-Length: 615
Content-Length: 615
< Connection: keep-alive
Connection: keep-alive
< Last-Modified: Tue, 25 Jan 2022 15:03:52 GMT
Last-Modified: Tue, 25 Jan 2022 15:03:52 GMT
< ETag: "61f01158-267"
ETag: "61f01158-267"
< Accept-Ranges: bytes
Accept-Ranges: bytes

< 
* Connection #0 to host app1.foo left intact


In [172]:
! kubectl logs app1 | tail

10.233.90.0 - - [10/Jun/2022:12:08:24 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.90.0 - - [10/Jun/2022:12:08:29 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.90.0 - - [10/Jun/2022:12:08:59 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.90.0 - - [10/Jun/2022:12:09:12 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.90.0 - - [10/Jun/2022:12:13:46 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.90.0 - - [10/Jun/2022:12:15:09 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.90.0 - - [10/Jun/2022:12:15:53 +0000] "GET / HTTP/1.1" 200 615 "-" "curl/7.68.0" "-"
10.233.92.2 - - [10/Jun/2022:12:27:04 +0000] "HEAD /1.html HTTP/1.1" 404 0 "-" "curl/7.74.0" "xxx.xxx.207.248"
2022/06/10 12:27:04 [error] 31#31: *20 open() "/usr/share/nginx/html/1.html" failed (2: No such file or directory), client: 10.233.92.2, server: localhost, request: "HEAD /1.html HTTP/1.1", host: "app1.foo"
10.233.92.2 - - [10/Jun/2022:12:27:22 

In [184]:
# get logs of first ingress pod on first node
! kubectl logs $(kubectl get po -n ingress-nginx -o jsonpath="{.items[0].metadata.name}") -n ingress-nginx | grep Event | tail -5

I0610 11:33:16.637822       6 event.go:285] Event(v1.ObjectReference{Kind:"Ingress", Namespace:"default", Name:"app1", UID:"23012dd3-7b59-45bb-adb9-e40e4503f86a", APIVersion:"networking.k8s.io/v1", ResourceVersion:"28796", FieldPath:""}): type: 'Normal' reason: 'Sync' Scheduled for sync
I0610 11:38:28.302832       6 event.go:285] Event(v1.ObjectReference{Kind:"Pod", Namespace:"ingress-nginx", Name:"ingress-nginx-controller-b2wcv", UID:"8dbc297b-96d7-425e-b4bf-7822c5981971", APIVersion:"v1", ResourceVersion:"4545", FieldPath:""}): type: 'Normal' reason: 'RELOAD' NGINX reload triggered due to a change in configuration
I0610 11:38:41.447946       6 event.go:285] Event(v1.ObjectReference{Kind:"Ingress", Namespace:"default", Name:"app1", UID:"32d1be7e-99d6-4be1-a43c-a59a99ce554d", APIVersion:"networking.k8s.io/v1", ResourceVersion:"29411", FieldPath:""}): type: 'Normal' reason: 'Sync' Scheduled for sync
I0610 11:38:41.535777       6 event.go:285] Event(v1.ObjectReference{Kind:"Pod", Namespa

In [183]:
# get logs of second ingress pod on second node
! kubectl logs $(kubectl get po -n ingress-nginx -o jsonpath="{.items[1].metadata.name}") -n ingress-nginx | grep Event | tail -5

I0610 11:33:16.637100       7 event.go:285] Event(v1.ObjectReference{Kind:"Ingress", Namespace:"default", Name:"app1", UID:"23012dd3-7b59-45bb-adb9-e40e4503f86a", APIVersion:"networking.k8s.io/v1", ResourceVersion:"28796", FieldPath:""}): type: 'Normal' reason: 'Sync' Scheduled for sync
I0610 11:38:28.310426       7 event.go:285] Event(v1.ObjectReference{Kind:"Pod", Namespace:"ingress-nginx", Name:"ingress-nginx-controller-z8qxk", UID:"ae08c28b-6ebd-4e37-9342-c98bcedb41da", APIVersion:"v1", ResourceVersion:"4573", FieldPath:""}): type: 'Normal' reason: 'RELOAD' NGINX reload triggered due to a change in configuration
I0610 11:38:41.447521       7 event.go:285] Event(v1.ObjectReference{Kind:"Ingress", Namespace:"default", Name:"app1", UID:"32d1be7e-99d6-4be1-a43c-a59a99ce554d", APIVersion:"networking.k8s.io/v1", ResourceVersion:"29411", FieldPath:""}): type: 'Normal' reason: 'Sync' Scheduled for sync
I0610 11:38:41.531107       7 event.go:285] Event(v1.ObjectReference{Kind:"Pod", Namespa

### Conclusion

I don't need an actual external loadbancer or metalLB to expose services publicly. 

Provisioning the ingress nginx controller and creating the Ingress assigns the public ip's of the node to the Ingress.

This means I can create an A record in a domain's zone I own to point to one of the two public node's ips. Even if the pod runs only on one node the requests gets routed to the pod.